In [ ]:
import pandas as pd

AMOUNT_OF_DATA = 160000

dataset = pd.read_csv(
    "data/final.csv"
)[:AMOUNT_OF_DATA].drop(columns=["id"]) # ID won't be usefull during training, so just dropping it.

print(dataset.head())

   store_nbr      family  sales  onpromotion  dcoilwtico   type_x type_y  \
0          1  AUTOMOTIVE    0.0            0         NaN  Holiday      D   
1          1   BABY CARE    0.0            0         NaN  Holiday      D   
2          1      BEAUTY    0.0            0         NaN  Holiday      D   
3          1   BEVERAGES    0.0            0         NaN  Holiday      D   
4          1       BOOKS    0.0            0         NaN  Holiday      D   

   cluster  
0       13  
1       13  
2       13  
3       13  
4       13  


In [51]:
from sklearn.model_selection import train_test_split

# Seperating the dataset into X, Y and then splitting that into a test and a train set 
X = dataset.drop('sales', axis=1)
y = dataset['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# The oil dataset is not completely filled, so we fill it in with the mean oil price
X_train['dcoilwtico'] = X_train['dcoilwtico'].fillna(X_train['dcoilwtico'].mean())
X_test['dcoilwtico'] = X_test['dcoilwtico'].fillna(X_train['dcoilwtico'].mean())

X_train = pd.get_dummies(X_train, columns=['family', 'type_x', 'type_y'], drop_first=True)
X_test = pd.get_dummies(X_test, columns=['family', 'type_x', 'type_y'], drop_first=True)

print(X_train.head())
print(X_test.head())

       store_nbr  onpromotion  dcoilwtico  cluster  family_BABY CARE  \
70164         28            0    94.32576       10             False   
15784         51            0    93.08000       17             False   
67672          8            0    95.84000        8             False   
97134         34            0    94.32576        6             False   
26570         54            0    93.26000        3             False   

       family_BEAUTY  family_BEVERAGES  family_BOOKS  family_BREAD/BAKERY  \
70164          False             False         False                False   
15784          False             False         False                False   
67672          False             False         False                False   
97134          False             False         False                False   
26570          False             False         False                 True   

       family_CELEBRATION  ...  family_POULTRY  family_PREPARED FOODS  \
70164                True  ... 

Another problem with the dataset is that some of the numbers are not between 0 and 1, which the program expects, welp... that's where the standardScaler comes in. It very simply just makes the numbers between them. We could write our own code for this, but the internet told me to do this, so I will.

In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
numerical_cols = ['dcoilwtico', 'onpromotion', 'store_nbr', 'cluster']
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])


# LinearRegression

Very simple linear model. I am not sure how it takes the class labels into account, but I assume it just sees each possible label as it's own axis (so in that case we would have 45 axis).

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Mean squared error: ", mean_squared_error(y_test, y_pred))
print("R2 score", r2_score(y_test, y_pred))

import matplotlib.pyplot as plt

plt.scatter(y_test, y_pred)
plt.xlabel("Actual sales")
plt.ylabel("predicted Sales")
plt.title("Linear Regression")
plt.plot([0, max(y_test)], [0, max(y_test)], "r--")
plt.show()


Mean squared error:  184462.67820427864
R2 score 0.5790034196055058


# K-nearest neighbour

So, since sklearn is actually really easy to use, this wasn't hard to implement. Just for fun I made a forloop that goes trough a few of the K values. 

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt

testList = [1, 3, 5, 7]

for i in testList:
    model = KNeighborsRegressor(n_neighbors=i)

    model.fit(X_train, y_train)

    predicted_values = model.predict(X_test)

    print("Mean_squared_error:", mean_squared_error(y_test, predicted_values))
    print("R2 score:", r2_score(y_test, predicted_values))

    plt.figure()
    plt.scatter(y_test, predicted_values)
    plt.xlabel("Actual sales")
    plt.ylabel("Predicted sales")
    plt.title(f"KNN with {i} neighbors")
    plt.plot([0, max(y_test)], [0, max(y_test)], "r--") # guideline thingy
    plt.show()

Mean_squared_error: 66777.09608786298
R2 score: 0.8475955712269779
Mean_squared_error: 49091.92859243429
R2 score: 0.887958180681418
Mean_squared_error: 48643.18065211797
R2 score: 0.8889823518046575
Mean_squared_error: 48238.035379106805
R2 score: 0.8899070091725387


# Decision Tree

A similar implementation to the ones above, just using a different model to see which ones suit best.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

model = DecisionTreeRegressor(max_depth=64, random_state=42)
model.fit(X_train, y_train)
predicted_values = model.predict(X_test)

print("Mean squared error: ", mean_squared_error(y_test, predicted_values))
print("R2 score: ", r2_score(y_test, predicted_values))

plt.figure()
plt.scatter(y_test, predicted_values)
plt.xlabel("Actual Sales")
plt.ylabel("Predicted Sales")
plt.title(f"Decision Tree (depth of 64)")
plt.plot([0, max(y_test)], [0, max(y_test)], "r--")
plt.show()


Mean squared error:  36291.506371652526
R2 score:  0.9171724045830517


# Random Forest

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=64, random_state=42)
model.fit(X_train, y_train)
predicted_values = model.predict(X_test)

print("Mean squared error: ", mean_squared_error(y_test, predicted_values))
print("R2 score: ", r2_score(y_test, predicted_values))

plt.figure()
plt.scatter(y_test, predicted_values)
plt.xlabel("Actual Sales")
plt.ylabel("Predicted Sales")
plt.title("Random Forest, 64 estimators")
plt.plot([0, max(y_test)], [0, max(y_test)], "r--")
plt.show()


Mean squared error:  33638.4962778328
R2 score:  0.9232273322688207


In [69]:

import numpy as np
from scipy import stats
import matplotlib.pyplot as plts

# MSE values for each model

mse_values = [
    # [178768.191, 185638.113, 162564.932, 201644.248, 189083.906],  # Linear Regression
    # [72236.518, 49957.147, 66318.39, 60139.697, 56238.727],   # KNN (k = 1)
    # [46854.559, 43213.887, 45083.834, 52086.285, 47649.818],   # KNN (k = 3)
    # [40370.251, 42420.620, 42557.732, 50782.059, 48704.903],   # KNN (k = 5)
    # [37602.404, 42221.357, 43185.170, 48782.908, 47929.983]   # KNN (k = 7)
    [31466.628, 28553.271, 36734.049, 35063.619, 36583.474],   # Random Forests
    [41658.577, 32363.860, 52799.208, 39940.704, 39306.239]    # Decision Trees
]
# avg = 0
# for model in mse_values:
#     for value in model:
#         avg += value
#     print(f"Average MSE: {avg/5}\n")
#     avg = 0

mse_results = stats.kruskal(*mse_values)
print("Kruskal-Wallis test results for MSE:")
print(f"H-statistic: {mse_results.statistic}, p-value: {mse_results.pvalue}")

# R-squared values for each model
r2_values = [
    # [0.581, 0.564, 0.582, 0.564, 0.557],  # Linear Regression
    # [0.830, 0.882, 0.829, 0.870, 0.868],  # KNN (k = 1)
    # [0.890, 0.898, 0.884, 0.887, 0.888],  # KNN (k = 3)
    # [0.905, 0.900, 0.890, 0.890, 0.886],  # KNN (k = 5)
    # [0.911, 0.900, 0.888, 0.894, 0.887]  # KNN (k = 7)
    [0.926, 0.933, 0.905, 0.924, 0.914],  # Random Forests
    [0.902, 0.924, 0.864, 0.913, 0.908]   # Decision Trees
]

# avg = 0
# for model in r2_values:
#     for value in model:
#         avg += value
#     print(f"Average r2: {avg/5}")
#     avg = 0

# Perform ANOVA
r2_results = stats.kruskal(*r2_values)
print("Kruskal-Wallis test results for R²:")
print(f"H-statistic: {r2_results.statistic}, p-value: {r2_results.pvalue}")

# mse_flat = [value for model in mse_values for value in model]
# r2_flat = [value for model in r2_values for value in model]


# def plot_distribution(data, title):
#     plt.figure(figsize=(12, 5))

#     # Histogram
#     plt.subplot(1, 2, 1)
#     plt.hist(data, bins=10, alpha=0.7, color='blue')
#     plt.title(f'Histogram of {title}')
#     plt.xlabel(title)
#     plt.ylabel('Frequency')

#     # Q-Q Plot
#     plt.subplot(1, 2, 2)
#     stats.probplot(data, dist="norm", plot=plt)
#     plt.title(f'Q-Q Plot of {title}')

#     plt.tight_layout()
#     plt.show()
# # Plot distributions
# plot_distribution(mse_flat, 'MSE Values')
# plot_distribution(r2_flat, 'R-squared Values')

# # Shapiro-Wilk Test for MSE
# shapiro_mse_stat, shapiro_mse_p = stats.shapiro(mse_flat)
# print(f'Shapiro-Wilk Test for MSE: Statistic={shapiro_mse_stat}, p-value={shapiro_mse_p}')

# # Shapiro-Wilk Test for R-squared
# shapiro_r2_stat, shapiro_r2_p = stats.shapiro(r2_flat)
# print(f'Shapiro-Wilk Test for R-squared: Statistic={shapiro_r2_stat}, p-value={shapiro_r2_p}')

friedman_mse_results = stats.friedmanchisquare(*mse_values)
print("Friedman test results for MSE:")
print(f"H-statistic: {friedman_mse_results.statistic}, p-value: {friedman_mse_results.pvalue}")

# Perform Friedman test for R²
friedman_r2_results = stats.friedmanchisquare(*r2_values)
print("Friedman test results for R²:")
print(f"H-statistic: {friedman_r2_results.statistic}, p-value: {friedman_r2_results.pvalue}")



Kruskal-Wallis test results for MSE:
H-statistic: 3.9381818181818176, p-value: 0.04720176769014221
Kruskal-Wallis test results for R²:
H-statistic: 2.8097560975609794, p-value: 0.09369261949324835


ValueError: At least 3 sets of samples must be given for Friedman test, got 2.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

dimensions = X_train.shape[1]

print(dimensions)

model_nn = Sequential([
    Dense(128, activation='relu', input_dim=dimensions),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)
])

model_nn.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model_nn.fit(X_train, y_train, validation_split=0.2, epochs=50, callbacks=[early_stopping], verbose=1)

nn_pred = model_nn.predict(X_test)

print("Neural Network MSE: ", mean_squared_error(y_test, nn_pred))
print("Neural Network R2: ", r2_score(y_test, nn_pred))

plt.figure()
plt.scatter(y_test, nn_pred)
plt.xlabel("Actual Sales")
plt.ylabel("Predicted Sales")
plt.title("Neural Network Predictions")
plt.plot([0, max(y_test)], [0, max(y_test)], "r--")
plt.show()

plt.gcf().savefig("neural_network_predictions.png")